# Импорт либ

In [ ]:
# !pip install transformers
# !pip install datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import torch.nn as nn
from transformers import BertTokenizerFast
from huggingface_hub import hf_hub_download
from transformers.modeling_outputs import SequenceClassifierOutput


## Шарю доступ к гугл диску

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Датафрейм

In [ ]:
file_path = "/content/drive/MyDrive/IMDB Dataset.csv"
df = pd.read_csv(file_path)
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})

## Трейн / тест сплит + датасеты/даталоадеры

In [ ]:
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42) #для сопоставимости резов внутри команды фиксируем 42 рандом сид и отрезаем 30 процентов на тест
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42) # теперь на трейне отрезаю тренировочный кусок и валидационный, пропорция 80/20
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        # принимаю фрейм с данными, токенизатор и максимальную длину последовательности
        self.data = dataframe.reset_index(drop=True)
        # сейвлю токенизатор для преобразования текста в токены
        self.tokenizer = tokenizer
        # запоминаю максимальную длину последовательности
        self.max_length = max_length

    def __len__(self):
        #  __len__ возвращает общее число примеров в фрейме
        return len(self.data)

    def __getitem__(self, idx):
        # __getitem__ позволяет получить конкретный пример по индексу

        # получаю текст отзыва по индексу
        review = self.data.iloc[idx]['review']
        # Получаю метку для этого отзыва - бинарнй таргет
        label = self.data.iloc[idx]['sentiment']

        # Токенизирую текст отзыва с помощью заданного выше предобученного токенизатора
        # padding паддинг до фиксированной длины
        # truncation : если текст длиннее макс длины, то он укорачивается
        # max_length: максимальная длина последовательности
        # return_tensors="pt": результат как торч тензор
        encoding = self.tokenizer(
            review,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # тк возвращаемые торч тензоры имеют размерность с дополнительным измерением,
        # я использую .squeeze(0), чтобы убрать это измерение и получить одномерные тензоры
        item = {key: tensor.squeeze(0) for key, tensor in encoding.items()}

        # + метка к моему словарю"labels", потому что сначала использовал импортнутый с библиотеки датасет и там
        # таргет назывался labels, поэтому немного костылей
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

train_dataset = IMDBDataset(train_df, tokenizer)
val_dataset = IMDBDataset(val_df, tokenizer)
test_dataset = IMDBDataset(test_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


Train size: 28000
Validation size: 7000
Test size: 15000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Переключаюсь на gpu

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Модели

## Сердюков Андрей

### Полный берт

In [ ]:
andrey_full_bert_tokenizer = BertTokenizer.from_pretrained("AndreySerdyukov/bert-imdb")
andrey_full_bert_model = BertForSequenceClassification.from_pretrained("AndreySerdyukov/bert-imdb").to(device)
andrey_full_bert_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Random Search

In [ ]:
random_search_tokenizer = BertTokenizer.from_pretrained("AndreySerdyukov/random-search")
random_search_model = BertForSequenceClassification.from_pretrained("AndreySerdyukov/random-search").to(device)
random_search_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-4): 5 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

### AlphaNas

In [ ]:
alphanas_tokenizer = BertTokenizer.from_pretrained("AndreySerdyukov/alphanas")
alphanas_model = BertForSequenceClassification.from_pretrained("AndreySerdyukov/alphanas").to(device)
alphanas_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

## Конзафаров Илья

### Полный берт

In [ ]:
hf_model_name = "ilkonz/all_imdb_bert"

ilya_full_bert_tokenizer = BertTokenizer.from_pretrained(hf_model_name)
ilya_full_bert_model = BertForSequenceClassification.from_pretrained(hf_model_name).to(device)
ilya_full_bert_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### AdaBert

In [ ]:
class FrozenAdaBERT(nn.Module):
    def __init__(self, vocab_size, hidden_size=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, 2)

    def forward(self, input_ids, **kwargs):
        x = self.embedding(input_ids)
        x = x.mean(dim=1)
        logits = self.classifier(x)
        return SequenceClassifierOutput(logits=logits)


adabert_tokenizer = BertTokenizerFast.from_pretrained("ilkonz/dnas")
model_file = hf_hub_download(repo_id="ilkonz/dnas", filename="frozen_adabert.pt")

adabert_model = FrozenAdaBERT(vocab_size=adabert_tokenizer.vocab_size).to(device)
adabert_model.load_state_dict(torch.load(model_file, map_location=device))
adabert_model.eval()

FrozenAdaBERT(
  (embedding): Embedding(30522, 256)
  (classifier): Linear(in_features=256, out_features=2, bias=True)
)

## Селиванец Алина

### Полный берт

In [ ]:
alina_full_bert_tokenizer = BertTokenizer.from_pretrained("alinaselivanets/bert-imdb")
alina_full_bert_model = BertForSequenceClassification.from_pretrained("alinaselivanets/bert-imdb").to(device)
alina_full_bert_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Bananas

In [ ]:
bananas_bert_tokenizer = BertTokenizer.from_pretrained("alinaselivanets/bananas-bert")
bananas_bert_model = BertForSequenceClassification.from_pretrained("alinaselivanets/bananas-bert").to(device)
bananas_bert_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

# Оценка нашей оптимизации

In [ ]:
def evaluate_models(models: dict, test_loader, device='cuda') -> pd.DataFrame:
    results = []

    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    for name, model in models.items():
        print(f"\n Оценка модели: {name}")
        model = model.to(device)
        model.eval()

        all_preds = []
        all_labels = []
        inference_times = []

        start_total = time.time()
        with torch.no_grad():
            for batch in tqdm(test_loader, desc=f"{name}"):
                inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
                labels = batch['labels'].to(device)

                start_time = time.time()
                outputs = model(**inputs)
                end_time = time.time()

                preds = torch.argmax(outputs.logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                inference_times.append(end_time - start_time)
        end_total = time.time()

        avg_batch_time = sum(inference_times) / len(inference_times)
        avg_sample_time_ms = avg_batch_time / test_loader.batch_size * 1000
        total_time_sec = end_total - start_total
        accuracy = accuracy_score(all_labels, all_preds)
        param_count = count_parameters(model)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Среднее время на пример: {avg_sample_time_ms:.2f} мс")
        print(f"Общее время инференса: {total_time_sec:.2f} сек")
        print(f"Параметров: {param_count:,}")

        results.append({
            "Модель": name,
            "Accuracy": round(accuracy, 4),
            "Время на пример (мс)": round(avg_sample_time_ms, 2),
            "Общее время (сек)": round(total_time_sec, 2),
            "Параметров": param_count
        })

    return pd.DataFrame(results)

In [ ]:
models_to_evaluate = {
    "BERT Андрей": andrey_full_bert_model,
    "BERT Илья": ilya_full_bert_model,
    "BERT Алина" :alina_full_bert_model,
    'BanaNAS' : bananas_bert_model,
    'Random Search NAS': random_search_model,
    'AdaBert' : adabert_model,
    'AlphaNAS' : alphanas_model
}

In [ ]:
df_results = evaluate_models(models_to_evaluate, test_loader)
df_results


 Оценка модели: BERT Андрей


BERT Андрей:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 0.9330
Среднее время на пример: 0.72 мс
Общее время инференса: 509.77 сек
Параметров: 109,483,778

 Оценка модели: BERT Илья


BERT Илья:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 0.9412
Среднее время на пример: 0.68 мс
Общее время инференса: 511.22 сек
Параметров: 109,483,778

 Оценка модели: BERT Алина


...

AdaBert:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 0.8801
Среднее время на пример: 0.02 мс
Общее время инференса: 74.20 сек
Параметров: 7,814,146

 Оценка модели: AlphaNAS


AlphaNAS:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 0.9027
Среднее время на пример: 0.30 мс
Общее время инференса: 209.73 сек
Параметров: 52,780,802


,Модель,Accuracy,Время на пример (мс),Общее время (сек),Параметров
0,BERT Андрей,0.9330,0.72,509.77,109483778
1,BERT Илья,0.9412,0.68,511.22,109483778
2,BERT Алина,0.9367,0.65,508.51,109483778
3,BanaNAS,0.9031,0.31,209.58,52780802
4,Random Search NAS,0.9077,0.35,246.48,59868674
5,AdaBert,0.8801,0.02,74.20,7814146
6,AlphaNAS,0.9027,0.30,209.73,52780802
